In [1]:
# import dependencies
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.preprocessing import StandardScaler
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import extract
import sqlite3 as sql
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'sqlalchemy'

In [2]:
# load data
db = sqlalchemy.create_engine('sqlite:///movies.sqlite')
imdb_df = pd.read_sql('select * from movies', db)
imdb_df

NameError: name 'sqlalchemy' is not defined

In [3]:
# open a connection to database
conn = sql.connect('movies.db')

NameError: name 'sql' is not defined

In [4]:
imdb_df.columns

NameError: name 'imdb_df' is not defined

In [5]:
# drop columns
imdb_df = imdb_df.drop(columns=['imdb_title_id', 'original_title', 'writer', 'production_company', 'actors','description'])
imdb_df

NameError: name 'imdb_df' is not defined

In [6]:
# drop nulls
imdb_df = imdb_df.dropna()

NameError: name 'imdb_df' is not defined

In [7]:
imdb_df

NameError: name 'imdb_df' is not defined

In [8]:
imdb_df['budget'] = pd.to_numeric(imdb_df['budget'].replace('[\DEM]','',regex=True))

NameError: name 'imdb_df' is not defined

In [9]:
imdb_df

NameError: name 'imdb_df' is not defined

In [10]:
# convert columns with $ to numeric
imdb_df[imdb_df.columns[11:12]] = imdb_df[imdb_df.columns[11:12]].replace('[\$]','',regex=True).astype(float)
imdb_df

NameError: name 'imdb_df' is not defined

In [11]:
imdb_df['worlwide_gross_income'] = pd.to_numeric(imdb_df['worlwide_gross_income'].replace('[\$]','',regex=True))

NameError: name 'imdb_df' is not defined

In [ ]:
imdb_df.head(10)

In [ ]:
# check unique values
imdb_df.nunique()

In [ ]:
imdb_df.dtypes

In [ ]:
meta_counts = imdb_df.metascore.value_counts()
meta_counts

In [ ]:
pd.cut(imdb_df['metascore'].astype(float),bins=4)

In [ ]:
# visualize the value counts of metascore
meta_counts.plot.density()

In [ ]:
# change metascore and title data types to float
imdb_df['metascore']=imdb_df['metascore'].astype(float)

In [ ]:
imdb_df.dtypes

In [ ]:
# create a new column 'hit'
def f(row):
    if float(row['metascore']) >= 75.0:
        val = '1'
    else:
        val = '0'
    return val

imdb_df['hit'] = imdb_df.apply(f, axis=1)

imdb_df

In [ ]:
# seperate the features(X) from the target (y)
y = imdb_df['hit']
X = pd.get_dummies(imdb_df.drop(columns='hit'))

# split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, stratify = y)
X_train.shape

In [ ]:
classifier = LogisticRegression(solver='lbfgs', max_iter = 200, random_state = 1)

In [ ]:
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)

In [ ]:
results.head(20)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

In [ ]:
report = classification_report(y_test, y_pred)
print(report)